In [1]:

import torch
import os
import sys
import numpy as np
sys.path.append('../src/')
from exttorch.models import Sequential
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.optim import Adam
from exttorch.metrics import Accuracy
import pandas as pd
from exttorch._data_handle import DataHandler

In [2]:
train_df = pd.read_csv('/Volumes/Storage/DS/DL/exttorch/datasets/digit-recognizer/train.csv')
test_df = pd.read_csv('/Volumes/Storage/DS/DL/exttorch/datasets/digit-recognizer/test.csv')

In [3]:
mask = np.random.randn(len(train_df)) <= 0.8
train_data = train_df[mask]
valid_data = train_df[~mask]
print(f"Train_data Shape {train_data.shape}\nValid_data: {valid_data.shape}")

Train_data Shape (33120, 785)
Valid_data: (8880, 785)


In [4]:
class CreateDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transforms: transforms = None,
                 train: bool =True, normalize: bool = True):
        self.transforms = transforms
        self.train = train
        self.df = df
        self.len = self.df.shape[0]
        self.normalize = normalize

    def __len__(self):
        return self.len

    def __getitem__(self, index: int):
        if 'label' in self.df.columns:
            image = self.df.iloc[index, 1:].values.reshape(28, 28)
            if self.normalize:
                image = self.df.iloc[index, 1:].values.reshape(28, 28) / 255
            label = self.df.iloc[index, 0]
            if self.transforms:
                image = self.transforms(image).type(torch.float)
            return image, label
        else:
            image = self.df.iloc[index, :].values.reshape(28, 28)
            if self.normalize:
                image = self.df.iloc[index, :].values.reshape(28, 28) / 255
            if self.transforms:
                image = self.transforms(image).type(torch.float)
            return image

In [5]:
# Construct Datasets train_X_set
train_X_set = CreateDataset(
    train_data,
    transforms=transforms.Compose([transforms.ToTensor()]))

# Construct valid_X_set dataset
valid_X_set = CreateDataset(
    valid_data,
    transforms=transforms.Compose([transforms.ToTensor()]))

# Construct test_data
test_data = CreateDataset(
    test_df,
    transforms=transforms.Compose([transforms.ToTensor()]))

In [6]:
BATCH_SIZE=64

In [7]:
# Coupling Datasets with dataloader function
train_dataloader = DataLoader(train_X_set, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_X_set, batch_size=BATCH_SIZE, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
model = Sequential([
    # Transpose Input data
    nn.Flatten(),

    # Input layer
    nn.Linear(in_features=28 * 28, out_features=256),
    nn.ReLU(),  # Activation function
    nn.Dropout(.4),  # Drop same pixel

    nn.Linear(in_features=256, out_features=256),
    nn.ReLU(),  # Activation function
    nn.Dropout(.4),  # Drop same pixel

    # Output layer
    nn.Linear(in_features=256, out_features=10),
    nn.Softmax(dim=1)
])

        

In [14]:
model.compile(
    optimizer=Adam(model.parameters()),
    loss=nn.CrossEntropyLoss(),
    metrics=[Accuracy()]
)

history = model.fit(train_dataloader, epochs=10, validation_data=valid_dataloader)

Epoch 1/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 45s 87ms/step - loss: 1.5547 - Accuracy: 0.9194 - val_loss: 1.5226 - val_Accuracy: 0.9399
Epoch 2/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 58s 112ms/step - loss: 1.5305 - Accuracy: 0.9360 - val_loss: 1.5106 - val_Accuracy: 0.9512
Epoch 3/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 71s 138ms/step - loss: 1.5197 - Accuracy: 0.9442 - val_loss: 1.5076 - val_Accuracy: 0.9542
Epoch 4/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 66s 128ms/step - loss: 1.5155 - Accuracy: 0.9494 - val_loss: 1.5015 - val_Accuracy: 0.9604
Epoch 5/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 74s 143ms/step - loss: 1.5063 - Accuracy: 0.9544 - val_loss: 1.5003 - val_Accuracy: 0.9611
Epoch 6/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 129s 249ms/step - loss: 1.5057 - Accuracy: 0.9554 - val_loss: 1.4987 - val_Accuracy: 0.9632
Epoch 7/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 108s 209ms/step - loss: 1.5031 - Accuracy: 0.9559 - val_loss: 1.4979 - val_Accuracy: 0.9636
Epoch 8/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 80s 154ms/step - loss: 1.5022 - Accuracy: 

In [15]:
print(history.history)

{'Accuracy': [0.9194, 0.936, 0.9442, 0.9494, 0.9544, 0.9554, 0.9559, 0.9594, 0.9617, 0.9633], 'val_Accuracy': [0.9399, 0.9512, 0.9542, 0.9604, 0.9611, 0.9632, 0.9636, 0.963, 0.961, 0.9668], 'loss': [1.5458, 1.5279, 1.5187, 1.5133, 1.5077, 1.5065, 1.5058, 1.5022, 1.4999, 1.4984], 'val_loss': [1.5226, 1.5106, 1.5076, 1.5015, 1.5003, 1.4987, 1.4979, 1.4979, 1.4999, 1.4949]}
